## Задание 1

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [231]:
link = 'https://habr.com/ru/all/'
KEYWORDS = ['python', 'парсинг','Роскосмос','Лауреат','туннелей']
times_,links_,titles_ = [],[],[]
def search_match(KEYWORDS, string):
    """
    Функция на вход принимает список слов и строку в которой надо искать. 
    Перебирает слова и возвращает True или False
    """
    for word in KEYWORDS:
        if word.lower() in string:
            return True
    return False

def search_full_text(link,key=KEYWORDS):
    """
    Если не нашли в превью, переходим по ссылке и ищем в полном тексте новости
    """
    req_1 = requests.get(link)
    time.sleep(0.2)
    html = BeautifulSoup(req_1.text, 'html.parser')
    txt = html.find('div', id = 'post-content-body')
    #Ищем совпадения с помощью функции search_match
    return search_match(key, txt.text)


In [232]:
def parse_habr(link,KEYWORDS):
    """
    Функция на вход получает ссылку и список ключевых слов, 
    после чего ищет пересечение в заголовках и текстах новостей
    """
    req = requests.get(link)
    soup = BeautifulSoup(req.text, 'html.parser')
    
    #парсим страницу с самыми свежими новостями и берем превьюхи
    preview = soup.find_all('article', class_ = 'post_preview')
    
    #Запускаем цикл для поиска в каждой новости
    for post in preview:
        #Достаем текст новости
        texts = post.find('div',class_ = 'post__text')
        #Достаем заголовок новости
        title = post.find('a', class_= 'post__title_link')
        #Время
        time = post.find('span', class_ = 'post__time')
        #Ссылка
        link_full = post.find('a', class_= 'post__title_link')
        link = link_full['href']
        
        #Проверяем с помощью функции search_match заголовок и текст новости в превью
        if (search_match(KEYWORDS, title.text.lower()) or search_match(KEYWORDS, texts.text.lower())):
#             print(title.text) для проверки
            
            #Добавляем в списки, если нашли
            times_.append(time.text)
            links_.append(link)
            titles_.append(title.text)
        #Если не находим в превью, переходим по ссылке и ищем в тексте   
        elif search_full_text(link,KEYWORDS):
            #print('full' + title.text) для проверки
            times_.append(time.text)
            links_.append(link)
            titles_.append(title.text)
        else:
            pass
           

In [233]:
parse_habr(link,KEYWORDS)
#Создаем датафрейм на основе списков
df = pd.DataFrame()
df['Время'] = times_
df['Ссылка'] = links_
df['Заголовок'] = titles_
df.head(30)




,Время,Ссылка,Заголовок
0,сегодня в 20:58,https://habr.com/ru/post/507608/,Визуализация при помощи генеративных алгоритмо...
1,сегодня в 16:59,https://habr.com/ru/company/piter/blog/522420/,"Нобелевский лауреат по (математике?) о «Моде, ..."
2,сегодня в 16:45,https://habr.com/ru/post/522396/,ipipou: больше чем просто нешифрованный туннель
3,сегодня в 16:32,https://habr.com/ru/company/fgts/blog/522432/,Удаленная работа или обзор VPN в Sophos XG Fir...


## Задание 2

In [147]:
EMAIL = ['eeee@e.ru', 'oooo@o.com']
link = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
req = requests.post(link, json = {'emailAddresses': EMAIL}, headers = headers)

In [148]:
# Создаем таблицу соответствия емейла и веток
xx = (req.json()['summary'])
mail, breaches = [],[]

#Добавляем в списки
for x,y in xx.items():
    for br in y['breaches']:
        mail.append(x)
        breaches.append(br)
        
#Создаем dataframe      
mail_br = pd.DataFrame()
mail_br['E-mail'] = mail
mail_br['breaches'] = breaches

#Переводим номера веток в строку, т.к. по умолчанию там np.int64
mail_br['breaches'] = mail_br['breaches'].apply(lambda x : str(x))

In [149]:
#Создаем вторую таблицу с описанием
breaches = pd.DataFrame(req.json()['breaches']).transpose().reset_index()
#Джойним таблицы
total = breaches.rename(columns={'index' : 'breaches'}).merge(mail_br, on='breaches', how = 'inner')

In [150]:
total.iloc[:, [-1,-2,-5,-3]]

,E-mail,publishDate,site,description
0,oooo@o.com,2019-11-14T00:00:00Z,toondo.com,"In July 2019, the online comic creation site T..."
1,oooo@o.com,2019-08-08T00:00:00Z,poshmark.com,"In May 2018, ""gently-used"" fashion retailer Po..."
2,oooo@o.com,2016-10-24T00:00:00Z,dropbox.com,Cloud storage company Dropbox suffered a major...
3,oooo@o.com,2017-05-04T00:00:00Z,uuu9.com,"On an unconfirmed date, UUU9's user database w..."
4,oooo@o.com,2017-06-01T00:00:00Z,taobao.com,"In January 2016, chinese online shopping site ..."
5,eeee@e.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
6,oooo@o.com,2017-04-14T00:00:00Z,17173.com,"On an unconfirmed date, Chinese gaming site 17..."
7,oooo@o.com,2017-03-01T00:00:00Z,heroesofnewerth.com,"In December 2012, the online MMOG Heroes of Ne..."
8,oooo@o.com,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
9,oooo@o.com,2020-09-10T00:00:00Z,na,This is a compilation of files including breac...
